In [1]:
import numpy as np
import pandas as pd

In [ ]:
def weighted_hs(rets, lam, window, p):
    """
    Compute the Weighted Historical Simulation Value-at-Risk (VaR).

    Parameters:
    rets (pd.Series): t-element pandas Series with a DatetimeIndex representing returns.
    lam (float): Smoothing parameter (0 < lam < 1).
    window (int): The size of the rolling window to compute the weighted CDF.
    p (float): The confidence level for VaR computation (e.g., 0.95 for 95% VaR).

    Returns:
    pd.Series: A series of 1-step ahead VaR forecasts with the first `window-1` values as NaN.
    """
    var_forecast = pd.Series(index=rets.index, dtype=float)
    
    # Compute weights
    weights = np.array([(1 - lam) * (lam ** i) for i in range(window)][::-1])
    weights /= weights.sum()  # Normalize to sum to 1
    
    for t in range(window - 1, len(rets)):
        window_data = rets.iloc[t - window + 1 : t + 1].values  # Get the rolling window data
        sorted_indices = np.argsort(window_data)  # Sort returns
        sorted_returns = window_data[sorted_indices]
        sorted_weights = weights[sorted_indices].cumsum()  # Compute cumulative weights

        # Find the VaR as the quantile where cumulative probability reaches (1 - p)
        var_forecast.iloc[t] = sorted_returns[np.searchsorted(sorted_weights, 1 - p)]
    
    return var_forecast

In [ ]:
def subsampled_rv(rets, k):
    """
    Compute the subsampled realized variance (RV).

    Parameters:
    rets (pd.Series): n-element pandas Series of observed high-frequency returns.
    k (int): The block size for subsampling.

    Returns:
    float: The subsampled realized variance.
    """
    n = len(rets)
    
    # Compute cumulative sums over rolling windows of size k
    subsampled_sums = rets.rolling(window=k).sum().dropna()
    
    # Compute the squared sum for each block
    squared_sums = subsampled_sums ** 2
    
    # Compute the subsampled realized variance
    rv = (n / k) / (n - k + 1) * squared_sums.sum()
    
    return rv